# Tutorial 9 (Introduction to AI)

# Genetic Algorithms

## Part 1: n-Queens with GAs

Inspired by genetic encoding and evolution, Genetic Algorithms (GAs) give a heuristic approach to optimisation problems.

A GA to solve an optimisation problem consists of:

* a collection of chromosomes, each chromosome being a list of numbers representing parameters which describe a candidate solution to the problem

* a fitness function, describing how good the candidate solution is

* a method to update the collection of chromosomes, the candidate solutions

To update the candidate solutions, the collection of chromosomes, the following are defined:

* crossover, a method to take two candidate solutions and produce one or more new solutions

* a method to select pairs of candidate solutions, making those with higher fitness more likely to be selected than those with lower fitness, often based on roulette wheel selection

* crossover rate, the proportion of pairs of candidate solutions to which crossover will be applied

* mutation, randomly changing an element of a chromosome to provide some variation in the solution space

* mutation rate, the rate at which mutation is applied (usually small)

* sometimes, tournament selection will be applied to select higher fitness candidate solutions for the next generation

* sometimes, a survival rate is applied, with the best fitting chromosomes being passed to the next generation

The GA picks an initial selection of chromosomes at random, then produces a new generation from this using the methods above.  This is run until a solution with maximum fitness has been found, or a fixed number of generations has passed.

## n-Queens

The n-Queens problem is to place n queens on an n x n chess board so that no piece can take another.   The number of potential configurations is large (it grows factorially), and the number of solutions for n is small in comparison.  A problem with a large search space is a good candidate for heuristic search.  A solution to the 8-queens problem can be seen below.

![8queens-min-conflicts.eps.jpg](attachment:8queens-min-conflicts.eps.jpg)

In this tutorial we will build a GA approach to the n-queens problem.

First, we will be using random numbers frequently.

Next, we define the GAs hyper-parameters.

In [ ]:
import random

nQueens = 8 #number of queens, size of board
population = 128 #the number of chromosomes to be maintained
pc = 0.7 #crossover rate
pm = 0.02 #mutation rate
generations = 100 #how many iterations to run
queens = [] #the collection of chromosomes

Often GAs are implemented with arrays of bits, but here the representation will be a list of integers representing position of a queen within its row.  For example, the solution to the 8-queens problem given above will be [4,0,7,3,1,6,2,5].  The following code initialise each chromosome (candidate solution).

In [ ]:
def initialise_chromosome():
    board = []
    for i in range(0,nQueens):
        board.append(random.randint(0,nQueens-1))
    return board

print(initialise_chromosome())

[4, 5, 4, 1, 5, 7, 5, 6]


Next we give a method to print a candidate solution, with 'Q' for queen, and '.' for empty square.

In [ ]:
#write chess board to terminal
def draw_queens(chromosome):
    for c in chromosome:
        for i in range(0,nQueens):
            if i==c:
                print('Q', end='')
            else:
                print('.', end='')
        print()

board = initialise_chromosome()
print(board)
draw_queens(board)

[2, 7, 3, 6, 1, 0, 7, 6]
..Q.....
.......Q
...Q....
......Q.
.Q......
Q.......
.......Q
......Q.


Fitness of a chromosome will be measured frequently.  Here, fitness is measure by the number of piece under threat.  For each queen we count how many pieces it threatens, and sum this for all queens.  Suppose that the number of threats is $n$, then the fitness is given by:
$$
\frac{1}{1+n}
$$
Hence, a solution has fitness $1$.

In [ ]:
def fitness(board):
    clashes = 0
    i=0
    while i < len(board):
        j = 0
        while j < len(board):
            if i!=j:
                if board[i]==board[j]:
                    clashes = clashes+1
                k = abs(i-j) #offset
                if board[i]+k <= nQueens:
                    if board[i] == board[j]-k:
                        clashes = clashes+1
                if board[i]-k >= 0:
                    if board[i] == board[j]+k:
                        clashes = clashes+1
            j=j+1
        i = i+1
    return 1/(1+clashes)

board = initialise_chromosome()
draw_queens(board)
print(fitness(board))

.....Q..
.......Q
....Q...
......Q.
Q.......
Q.......
.......Q
....Q...
0.1111111111111111


Fitness is not always on a [0,1] scale as it is here.  Hence a probability method, which in this instance is just a wrapper to fitness, is included here.

The random_pick method implements roulette wheel selection.  It sums up the probabilities (fitness) of each element in the population, picks a random number in this interval and then moves through the population until the element corresponding to this number is found.  This means that the chromosome with better fitness are more likely to be selected.

In [ ]:
def probability(board):
    return fitness(board)

def random_pick(population, probabilities):
    populationWithProbabilty = zip(population, probabilities)
    total = sum(w for c, w in populationWithProbabilty)
    r = random.uniform(0, total)
    upto = 0
    for c, w in zip(population, probabilities):
        if upto + w >= r:
            return c
        upto += w

Crossover is implemented as below, returning the two crossed-over boards.

In [ ]:
def crossover(board1, board2, i):
    new_board1 = board1[0:i]+board2[i:]
    new_board2 = board2[0:i]+board1[i:]
    return new_board1, new_board2

board1 = initialise_chromosome()
board2 = initialise_chromosome()
print(board1)
print(board2)
cross1, cross2 = crossover(board1, board2, 3)
print(cross1)
print(cross2)

[6, 1, 7, 5, 5, 3, 1, 3]
[4, 7, 2, 7, 5, 7, 6, 1]
[6, 1, 7, 7, 5, 7, 6, 1]
[4, 7, 2, 5, 5, 3, 1, 3]


Mutation is here applied at an element by element level.  So each element of a chromosome is considered in turn, and randomly considered for mutation.  Here mutation means replacing with a randomly generated alternative.

In [ ]:
def mutation(board):
    i=0
    while i<len(board):
        r = random.random()
        if r<pm:
            board[i]=random.randint(0,nQueens-1)
        i=i+1
    return board

board = initialise_chromosome()
print(board)
mut_board = mutation(board)
print(mut_board)

[3, 2, 3, 6, 2, 1, 0, 4]
[3, 2, 3, 6, 2, 1, 0, 4]


We generate an initial population using a series of calls to initialise_chromosome.

In [ ]:
def initial():
    for i in range(0,population):
        queens.append(initialise_chromosome())

The following method gives the next generation.  

* After calculating the probabilities (just fitness here), two chromosomes are picked using the roulette wheel selection.

* A random number is compared to the crossover rate to determine whether or not to apply crossover

* Mutation is called

* The results are appended to the next generation, but notice that this gives a doubling of the population size

* The final step is to use tournament selection to half the number of chromosome (that is to give the same size population as before).  The next_gen list is shuffled, then considered two by two, with the chromosome with highest fitness making it to the final_next_gen returned.

In [ ]:
def next_gen():
    next_gen = []
    probabilities = [probability(board) for board in queens]
    for i in range(population):
        x = random_pick(queens, probabilities)
        y = random_pick(queens, probabilities)
        rand = random.uniform(0,1)
        if rand < pc:
            p = random.randint(1,nQueens-1)
            c1,c2 = crossover(x,y,p)
        else:
            c1,c2=x,y
        m1 = mutation(c1)
        m2 = mutation(c2)
        next_gen.append(m1)
        next_gen.append(m2)
    random.shuffle(next_gen)
    final_next_gen = []
    j=0
    while j<len(next_gen):
        if fitness(next_gen[j])>fitness(next_gen[j+1]):
            final_next_gen.append(next_gen[j])
        else:
            final_next_gen.append(next_gen[j+1])
        j=j+2
    return final_next_gen

To bring that together, let's find solutions.  The population is initialised, and solution set to an empty list.  next_gen is invoked to find a new population until a solution with fitness 1 is found, or the number of iterations is exhausted.

In [ ]:
queens = [] #to start with a fresh population
initial()
soln=[]
for i in range(0,generations):
    print("generation", i, sep=" ")
    queens = next_gen()
    for queen in queens:
        if fitness(queen)==1.0:
            soln=queen
            break
    if soln !=[]: break

if soln==[]:
    soln=queens[0]
print(soln)
draw_queens(soln)
f = fitness(soln)
print(f'fitness {f}')

generation 0
generation 1
[6, 2, 7, 1, 4, 0, 5, 3]
......Q.
..Q.....
.......Q
.Q......
....Q...
Q.......
.....Q..
...Q....
fitness 1.0


If you run this for 8 queens you will find that you typically find a solution in 10-15 or so generations, but that this is quite variable, with some runs even failing to find a solution at all.

You should try varying the parameters to find bigger solutions (e.g. 16 queens with population 256, or 40 with 512 and 1000 generations).  You might need more than one run to find a solution.  You might also experiment with changing the pc and pm values.